In [236]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [173]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_74056/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [269]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)

0 2035


In [270]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list = []
risk_list = []

# instantiate a new Session 
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
           'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
          }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

counter = 0
block_number = 0

# pause for 5 seconds every 5 iterations
for i in range(block_number * block_size, (block_number + 1) * block_size):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(60)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
              'content-type': 'application/json',
              'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
              'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
              'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list.append("")
        risk_list.append("")
        continue

--------ademibar/ademi-plant-based-protein-bar------
-------STORY--------
Ademi Bar: Our Journey
While looking back and thinking about where our journey started, we realized that the story of every entrepreneur and of every immigrant in the United States is very similar, it is the story of unimaginable hard work and never ending efforts to make it work. It was no different for us.
Here we were in 2015, the newly immigrated and entrepreneurial just-married couple. I, Aizhan, think that this is where our journey had begun… Aidar thinks our journey started when we said NO to meat, but that is just a few months later, so let's not think about little differences but common points.
It was 2015, when we started experimenting with food and trying out cooking for people. As new folks in town (country), we thought sharing food is the fastest way to find like minded friends which eventually turned out to be a good plan. Please note this for yourselves if any of you happen to move to a new place w

-------STORY--------
 Our company Unicox Superfoods, 
    Power Blend is for everyone,
   who are looking for performance breakfast,
   our product is a set of Nutrition for your mind and body,
   that's made up of Clean and organic ingredients.
And, We believe that together we can change the food industry..
 Here's how our macro-nutritional profile stacks up with other Breakfasts:
That's why Unicox Power Breakfast are packed with healthy fats and contain just 4-5g of net carbs per serving. We also included 6g of fiber/per serving to slow the absorption of those net carbs...
To bring all this to life I mastered recipe at home and with help of  nutritionist and RD's we fixed caloric profile, Our blend have a "fat-fueled" mentality. They're packed with healthy fats from chia seeds, flax seeds and almonds to provide the ultimate form of energy and naturally sweetened with monk fruit. We developed them from the ground-up to be both functional and delicious.
“Breakfast is the most important

-------STORY--------
In our society, many issues exist that people live with as part of their day to day lives.  Such issues include discrimination (race, gender, etc.) rape, sexual assault, abuse, bullying, poverty, body image, suicide, depression, and so many more.  As someone who was bullied for most of his schooling career, I feel very strongly about raising awareness of these many issues. This is a project I have wanted to do for years, but such a major undertaking is of course, very expensive.  To make this project a reality, I need more than just a single camera.  I need a studio so that I can mold my photos into what I want. Equipment, such as lights, video cameras, and other items are essential. Finally, it's necessary to hire models or pay to bring survivors to me so that they can be a part of this project.  This is no small idea.That's where you come in.  I have created a KickStarter page for this project. My goal is not just to do this for myself, but to raise awareness on 

--------1869715485/still-alive-1------
-------STORY--------
Tired of slow boring zombie movies that nothing ever happens in, and zombies and "terrifying" events are a joke?   This movie is a non stop action never stop run through the city in terror, in a doomed apocalypse  where everyone will die.  No one will be left after this, the only goal is to stay alive as long as possible.  If you fall asleep, or sit still and let your heart rest, you turn into a zombie.  Think Crank crossed with the living dead, but scary and horror filled.   Humanity will tear itself apart. 
-------RISKS--------
I'll need to hire a writer, make up artists and put together a film crew and editors/sfx, all for a relatively small amount.  This will be tough to come up with skilled people and basically this is the first time I've done anything like this, so this movie will probably suck. lol  So many people have tried something like this and failed, zombie movies are a dime a dozen, but there are so few good ones

--------cosmojoe/sci-fi-art-calendar-w-meteor-showers------
-------STORY--------
With: Holidays, Full Moons, Solstice, Equinox, Eclipse, and Meteor Showers!
Don't miss another Meteor Shower or Eclipse!!!
"This is the 3rd year that I am making a calendar. Previous years I only offered them to family, friends, and fans. This is the first year that anyone can get one." - Cosmo Joe
Or go BIG and get one of 2 paintings featured in the calendar. Many people will have the artwork on the calendar but ONLY YOU will have the original.
Get a $300 painting AND the calendar AND free shipping when you get one of the 2 paintings below but be quick! They will be gone soon.
-------RISKS--------
This will be the 3rd Cosmo Joe calendar so there should not be any problems. If there is a problem with the printer we will just use someone else.
--------nexveg/nexveg-100-whole-food-plant-based-non-allergen-proteins------
-------STORY--------
While it would be a set-back to keeping it local, we are already sto

...

-------STORY--------
Why Paint 100 Blue Face Paintings & Give them Away?
Hi there. I'm Anna aka Bunny Glue. I'm an artist based in Vancouver, BC, Canada.
Back in February I painted a blue face in acrylic on canvas. And then another. And just one more. Then I said to a friend, "I'm going to paint one hundred of these." She shrugged and said, "You're crazy." And so I painted until I dreamed blue faces. And it was amazing. 
By June 25th I had finished painting one hundred. Now I have 100 paintings that need homes. Couches to live over. Other people to admire them. And I need money to buy more paint and canvas.
They need homes.
So here's what I decided to do:
1. Give away 50 Paintings. 
2. Swap 20 paintings for Karma.
3. Make a little pocket change (aka paint & canvas money) from choice.
It's not as complicated as it sounds. 
Shipping to US/Canada included in basic shipping pricing. I'm happy to ship internationally, just contact me and I can figure out any additional cost based on your lo

-------STORY--------
 Who doesn't love pinnipeds?! These 4 pins represent some of our favorite mammals, whose survival all depend on the oceans we love. Okay, so they aren’t ALL pinnipeds but, for puns sake we are rolling with it! We hope you wear/display our pins as talking points for these species! As always, wear something that means something. <3
If our project is successfully funded, we will send out reward surveys to our backers! The reward surveys will allow you to choose your pins! :) If our project is not funded to the $500 goal, you will not be charged and we will not receive funds for this project.
This Kickstarter's initial funding goal is $500 USD. This will help us produce all 4 hard enamel pins as well as help to cover backing cards, Kickstarter fees, taxes, and shipping costs/supplies. Though $500 USD doesn't cover all the fees, it will help us off to a great start! 
Once the Kickstarter has ended and backer surveys are completed, all pins will go on to production! Prod

-------STORY--------
To start out 2020 I have a trio of unrelated cat designs.  I have several larger more cohesive 'collections' that I want to release this year, so solo pins like these either need to have very short KS campaigns or be grouped together like this.  (I have another unrelated design that will be getting the one to two week treatment.)
I completely understand that these may have different audiences so don't feel bad if you are only interested in one of these designs!
My kids love for me to tell bedtime stories.  These stories often star their stuffed animals.  They have this little set of kitties that they love dearly.  One multi-chapter story involved a ninja kitty who trained their kitties as ninja so that they could team up and help him defeat an evil sorcerer. I designed 5 separate Ninja Kitties, but have chosen to have Snowflake as a ninja made into a pin.  Who knows, if this one is well received I may try to do the others at a later date. 
If you have ever simultan

--------hannahthomas/truth-about-an-outlaw-ep------
-------STORY--------
I am super excited about this project and I really want to get these songs out to you. I have enlisted the help of some great musicians and these are the songs you have been asking for. 
1. Truth About an Outlaw  2. (God Help My Mama) I Turned  Out Like My Dad  3. Let It Rain 4. Lie to Me
I will be working with Don Dixon producing and playing bass. Jim Brock on percussion, Tomi Martin and Cooper Carter on guitar.
With your help we can get these songs out to the world. 
-------RISKS--------
The biggest challenge to getting the record out by September will be making sure the band can all be off for 3 days to make the record.

The other challenges that we can forsee we have found help to package envelopes, so that should be covered.

This will be 4 songs instead of the 8 we were looking to make in the prior kickstarter project. We have a new studio on board to help us make the project that will make this go quickly a

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [280]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "0_99_story"
risk_file_name = "0_99_risk"

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

["Ademi Bar: Our Journey While looking back and thinking about where our journey started, we realized that the story of every entrepreneur and of every immigrant in the United States is very similar, it is the story of unimaginable hard work and never ending efforts to make it work. It was no different for us. Here we were in 2015, the newly immigrated and entrepreneurial just-married couple. I, Aizhan, think that this is where our journey had begun… Aidar thinks our journey started when we said NO to meat, but that is just a few months later, so let's not think about little differences but common points. It was 2015, when we started experimenting with food and trying out cooking for people. As new folks in town (country), we thought sharing food is the fastest way to find like minded friends which eventually turned out to be a good plan. Please note this for yourselves if any of you happen to move to a new place with no friends or family. We were cooking different sweets and snacks, s

In [273]:
len(risk_list)
len(story_list)

100

In [278]:
df_1.iloc[99]['blurb']

'My wife and I out of improvisation created these nourishing and incredibly satisfying Pizzas we wanna share with the world in any way!'

In [165]:
s = requests.Session()

In [234]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [256]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "operationName": "Campaign",
    "variables": {
      "slug": "leightonconnor/akashic-titan-blue-bolt"
    },
    "query": "query Campaign($slug: String!) {\n  project(slug: $slug) {\n    id\n    isSharingProjectBudget\n    risks\n    story(assetWidth: 680)\n    currency\n    spreadsheet {\n      displayMode\n      public\n      url\n      data {\n        name\n        value\n        phase\n        rowNum\n        __typename\n      }\n      dataLastUpdatedAt\n      __typename\n    }\n    environmentalCommitments {\n      id\n      commitmentCategory\n      description\n      __typename\n    }\n    __typename\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'GgpaCc5tY-2VRVj1bE1aLOPKLkwRFEbIF205o2WorHLvaUEoOIPiZJSx4rbQg0csVbwBfq-jKGXPw2j42dvQCQ',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
  'cookie': '_ksr_session=aVeEJvdlSYzJIE8X7f3v85C2T7b9Q30EN0ONWdeWjMLei6H06q6eRA3DZfCsbal40AfBTSbfczmlTv2ZIpvdyykn4QQYZdSlIzlalRivb6Xlb%2BXhXDhBNJg7KPYEEyFFIX3Qz8%2F61YNIoV136X5ju%2BE7jrqQFsGEB1%2FhAwJZ7kjD6QiwTDFdWWmJDr2wmREspLGXKUO0gicnsrUxQJPdTQMCcFNXy6sx95x5S%2FiNs4xVq0t4jWv%2BaiSnwBp8pC6Xu81FBhpD%2F9sNLZ9YyS5Obzbv0jI%3D--tM1UdSPo8XKxtj%2Bt--WQhnQX4EovqKwzBU9tiWIg%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json()[0]['data']['project']['risks'])

200
This is my fourth solo Kickstarter campaign, and I delivered all three of my previous games in a timely manner. 

I have a complete draft of the text, with the exception of Appendix 1. I have a team of editors lined up who will give me feedback so that I can revise, expand, and polish it as needed. The most time-consuming tasks will be completing the artwork and the layout. I've done this before, with the previous Akashic Titan zine and with my game Project 8Ball. Depending on the stretch goals I will be depending on someone else for illustrations and additional text, but I have worked with Josh before and I am confident he will hit his deadlines. 

I believe I will be able to finish the book by June, but just to be cautious I have set the estimated delivery month as July. 

Barring any unforeseen catastrophes, I should have no problem printing and shipping the zine by July.


In [255]:
s = requests.Session()
headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
          }
# GET request to kickstarter to get some token needed for later 
r = s.get("https://www.kickstarter.com", headers=headers)
#BeautifulSoup parses the html returned as a response from the GET request 
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

In [ ]:
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'

In [237]:
print("hello")
time.sleep(3)
print("goodnight")

hello
goodnight
